In [ ]:
import pandas as pd
import requests
import psycopg2
import time
import re
import chromedriver_autoinstaller
from datetime import datetime
from psycopg2 import Error
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

In [ ]:
# 중고나라 코드 -> 링크를 통해서 상품번호 받아오기
def search_link(keyword):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    df = pd.DataFrame(columns=['keyword', 'link'])
    link_list = []

# 페이지수 
    for i in range(1, 2):
        try:
            driver = webdriver.Chrome(options=chrome_options)
            url = f'https://web.joongna.com/search/{keyword}?page={i}'
            driver.get(url)
            driver.maximize_window()

            href_element = driver.find_element(By.XPATH, '//*[@id="__next"]/div/main/div[1]/div[2]/ul')
            text = href_element.get_attribute('innerHTML')

            pattern = r'href="([^"]*)"'
            href_tags = re.findall(pattern, text)
            for href in href_tags:
                if href.startswith("/product/"):
                    link = 'https://web.joongna.com' + href
                    id = link.split('/')[-1]
                    link_list.append(id)

            driver.quit()
            time.sleep(2)

        except:
            print("end")
            pass

    df['link'] = link_list
    df['keyword'] = keyword
    return df


In [ ]:
# 중고나라 데이터 추출
def restaurant_extract(link):
    url = f'https://web.joongna.com/_next/data/lYle0daistM70s0JL0yaB/product/{link}.json?productSeq={link}' #/data/ 뒤에 코드는 맨날 확인해서 변경
    response = requests.get(url)
    now = datetime.now()

    try:
        data = response.json().get('pageProps', {}).get('dehydratedState', {}).get('queries', [{}])[0].get('state', {}).get('data', {}).get('data', {})
        product_media = data.get('productMedia', [])
        viewOption = data.get('viewOption', {})
        sortdate = data.get('sortDate')

        if sortdate:
            sort_date = datetime.strptime(sortdate, '%Y-%m-%d %H:%M:%S')
            times = (now - sort_date).total_seconds() / 60
            if times < 60:
                timess = f"{round(times)} 분 전"
            else:
                timess = f"{round(times / 60)} 시간 전"
        else:
            timess = ''

        images = []
        for media in product_media:
            image = {
                'originUrl': media.get('originUrl', ''),
                'mediaUrl': media.get('mediaUrl', ''),
                'thumbnailUrl': media.get('thumbnailUrl', ''),
                'waterMarkUrl': media.get('waterMarkUrl', ''),
                'waterMarkLogoUrl': media.get('waterMarkLogoUrl', ''),
                'mediaSort': media.get('mediaSort', 0)
            }
            images.append(image)

        return {
            'name': data.get('productTitle', ''),
            'description': data.get('productDescription', ''),
            'price': data.get('productPrice', ''),
            'time': timess,
            'favoriteCount': viewOption.get('wish', {}).get('wishCount', ''),
            'viewCount': data.get('viewCount', ''),
            'images': images,
            'address': '',
            'keyword': keyword,
            'crawling': '중고나라'
        }
    except (requests.exceptions.JSONDecodeError, ValueError):
        return {}

